# Simulate a user accessing scenes to estimate # of sectors needed to unseal

## Imports

In [1]:
import random
import geopandas as gpd

## Prepare Landsat scenes (Rows/Paths) of the US

## Fetch files

In [2]:
# From https://www.usgs.gov/media/files/landsat-wrs-2-descending-path-row-shapefile
landsat_scenes_shp = gpd.read_file("https://d9-wret.s3.us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/WRS2_descending_0.zip")

# From https://www.census.gov/geographies/mapping-files/time-series/geo/carto-boundary-file.html
us_states_shp = gpd.read_file("https://www2.census.gov/geo/tiger/GENZ2018/shp/cb_2018_us_state_20m.zip")

## Reproject to WGS84 (to match Landsat)

In [3]:

us_states_shp = us_states_shp.to_crs(epsg=4326)

In [4]:
us_states_continential = us_states_shp[us_states_shp["STUSPS"].isin(["AK", "HI", "PR", "VI", "GU", "AS", "MP", "UM"]) == False]
landsat_scenes_clipped = gpd.clip(landsat_scenes_shp, us_states_continential)
landsat_scenes_clipped # Should be about 457 scenes

/Users/jsolly.admin/Documents/code/easier/venv/lib/python3.10/site-packages/geopandas/tools/clip.py:67: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  clipped.loc[


In [6]:
landsat_scenes_clipped_rows_paths = landsat_scenes_clipped[["PATH", "ROW"]]

## Prepare US Metropolitan Statistical Areas (MSAs)

In [ ]:
US_cell_dict = {
    cell_1: {row: 1
             path: 1
             sector: 1}
    cell_2: {row: 30
             path: 30
             sector: 2}
}

# The number of Landsat scenes intersected by MSA
MSA_dict = {
    'MSA1': ['cell1', 'cell2', 'cell3'],
    'MSA2': ['cell4', 'cell5'],
}

## User Access Settings

In [ ]:
scenes_requested = random.randint(1, 5)

## Sector and Cache Settings

In [ ]:
hot_storage_cache_time = 1000 # Number of requests (loops) before hot storage cache is cleared
